In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime as dt
import os

def getNow():
    now       = dt.now().strftime('%Y-%m-%d--%H-%M-%S')
    nowFolder = os.path.join('./logFolder', now)
    return nowFolder

In [2]:
N = 1000
X = np.random.rand(N, 2)
y = 2*X[:, 0] + 3*X[:, 1] + 4 + 0.1*(np.random.rand(N)-0.5)
y = y.reshape((-1, 1))
tMask = np.random.choice([True, False], (N,), p=[0.8, 0.2])

In [3]:
with tf.variable_scope('trainData'):
    inpTrain = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='inpTrain')
    outTrain = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='outTrain')

with tf.variable_scope('testData'):
    inpTest = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='inpTest')
    outTest = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='outTest')

with tf.variable_scope('model'):
    W = tf.Variable(initial_value=np.random.rand(2, 1), name='W', dtype=tf.float32)
    b = tf.Variable(initial_value=0, name='b', dtype=tf.float32)
    
    # Add summaries 
    # ---------------------
    tf.summary.scalar('bSummary', b)
    tf.summary.histogram('WSummary', W)
    tf.summary.scalar('WMin', tf.reduce_min(W))
    tf.summary.scalar('WMax', tf.reduce_max(W))

    with tf.variable_scope('train'):
        yHatTrain = tf.matmul(inpTrain,  W )+ b
        errTrain  = tf.reduce_mean((outTrain - yHatTrain)**2, name='errTrain')
        tf.summary.scalar('trainErr', errTrain)
        
    with tf.variable_scope('test'):
        yHatTest = tf.matmul(inpTest,  W )+ b
        errTest  = tf.reduce_mean((outTest - yHatTest)**2, name='errTest')
        tf.summary.histogram('testErrHist', (outTest - yHatTest)**2)
        tf.summary.scalar('testErr', errTest)

with tf.variable_scope('optimizer'):
    lr = tf.Variable(0.01, dtype=tf.float32, name='learningRate')
    opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(errTrain)

init = tf.global_variables_initializer()

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
merged = tf.summary.merge_all()

In [5]:
folder = getNow()

with tf.Session() as sess:
    writer = tf.summary.FileWriter(folder, sess.graph)
    sess.run(init)
    
    for i in range(5001):
        sess.run( opt, feed_dict={
            inpTrain : X[tMask, :],
            outTrain : y[tMask, :],
            lr       : 0.01
             })
        
        if (i % 100) == 0:
            summary, errTrainVal, errTestVal = sess.run( [merged, errTrain, errTest], feed_dict={
                inpTrain : X[tMask, :],
                outTrain : y[tMask, :],
                inpTest : X[~tMask, :],
                outTest : y[~tMask, :]})
            
            writer.add_summary(summary, i)
            print('Training: {}, Testing: {}'.format(errTrainVal, errTestVal))
            
writer.close()

print('Final Values')
print('-'*30)
print('Training: {}, Testing: {}'.format(errTrainVal, errTestVal))
    
print('tensorboard --logdir={}'.format( folder ))

Training: 32.92376708984375, Testing: 32.44841384887695
Training: 14.910260200500488, Testing: 14.674463272094727
Training: 5.778097152709961, Testing: 5.6680121421813965
Training: 1.9550249576568604, Testing: 1.9012222290039062
Training: 0.6636378169059753, Testing: 0.6317040324211121
Training: 0.3111761510372162, Testing: 0.2873738706111908
Training: 0.2244434356689453, Testing: 0.20431579649448395
Training: 0.19500388205051422, Testing: 0.1773156225681305
Training: 0.17522749304771423, Testing: 0.15963959693908691
Training: 0.1571352332830429, Testing: 0.14350537955760956
Training: 0.13989104330539703, Testing: 0.1280975490808487
Training: 0.1235775351524353, Testing: 0.11348500102758408
Training: 0.10832385718822479, Testing: 0.09978289157152176
Training: 0.09422020614147186, Testing: 0.08707331120967865
Training: 0.081316277384758, Testing: 0.07540386915206909
Training: 0.06962781399488449, Testing: 0.06479344516992569
Training: 0.05914236605167389, Testing: 0.055236417800188065
T